<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/DenseNet169.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 24.3 MB/s 
     |████████████████████████████████| 182 kB 73.5 MB/s 
     |████████████████████████████████| 166 kB 65.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 68.8 MB/s 
     |████████████████████████████████| 162 kB 72.3 MB/s 
     |████████████████████████████████| 162 kB 59.2 MB/s 
     |████████████████████████████████| 158 kB 48.2 MB/s 
     |████████████████████████████████| 157 kB 63.8 MB/s 
     |████████████████████████████████| 157 kB 53.5 MB/s 
     |████████████████████████████████| 157 kB 52.4 MB/s 
     |████████████████████████████████| 157 kB 67.0 MB/s 
     |████████████████████████████████| 157 kB 59.8 MB/s 
     |████████████████████████████████| 157 kB 56.7 MB/s 
     |████████████████████████████████| 157 kB 65.5 MB/s 
     |████████████████████████████████| 156 kB 62.7 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
densenet121 = tf.keras.applications.DenseNet169(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)
model_densenet121=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  densenet121,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
densenet121.trainable=False
model_densenet121.summary()

58541896/58541896 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 densenet169 (Functional)    (None, 1000)              14307880  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 5)                 5005      
                                                                 
Total params: 14,312,885
Trainable params: 5,005
Non-trainable params: 14,307,880
_________________________________________________________________


In [7]:
model_densenet121.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [8]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "densenet169"
      )

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [9]:
%%time
epochs=40
history = model_densenet121.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 1.5994 - accuracy: 0.3986

wandb: Adding directory to artifact (/content/wandb/run-20221106_050630-3308klzv/files/model-best)... Done. 0.4s


19/19 [==============================] - 171s 6s/step - loss: 1.5994 - accuracy: 0.3986 - val_loss: 1.5865 - val_accuracy: 0.4865
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 1.5778 - accuracy: 0.4949

wandb: Adding directory to artifact (/content/wandb/run-20221106_050630-3308klzv/files/model-best)... Done. 0.4s


...

19/19 [==============================] - 66s 4s/step - loss: 1.1644 - accuracy: 0.5135 - val_loss: 1.1857 - val_accuracy: 0.4797
Epoch 39/40
19/19 [==============================] - ETA: 0s - loss: 1.1592 - accuracy: 0.5220

wandb: Adding directory to artifact (/content/wandb/run-20221106_050630-3308klzv/files/model-best)... Done. 0.6s


19/19 [==============================] - 66s 4s/step - loss: 1.1592 - accuracy: 0.5220 - val_loss: 1.1797 - val_accuracy: 0.4865
Epoch 40/40
19/19 [==============================] - ETA: 0s - loss: 1.1521 - accuracy: 0.5186

wandb: Adding directory to artifact (/content/wandb/run-20221106_050630-3308klzv/files/model-best)... Done. 0.4s


19/19 [==============================] - 66s 4s/step - loss: 1.1521 - accuracy: 0.5186 - val_loss: 1.1738 - val_accuracy: 0.5000
CPU times: user 42min 26s, sys: 1min 17s, total: 43min 44s
Wall time: 51min 23s


In [10]:
wandb.finish()

accuracy,▁▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▇▆▆▆▇▇▆▇▇▇▇▇█▇▇▇████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁
val_accuracy,▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▄▄▄▄▄▅█
val_loss,██▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
accuracy,0.51858
best_epoch,39
best_val_loss,1.17377
epoch,39
loss,1.15211
val_accuracy,0.5


In [11]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
